In [1]:
import platform
print('Python Version ->', platform.python_version())
import findspark
findspark.init()
findspark.find()
import pyspark
print('Spark Version ->', pyspark.__version__)

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf, SparkFiles
from datetime import *

Python Version -> 3.7.13
Spark Version -> 2.4.6


In [2]:
appName = "ColetaRaw_Cov_Vac"
spark = SparkSession \
        .builder \
        .appName(appName) \
        .master('local[*]')\
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.11:2.2.1")\
        .config("spark.mongodb.input.uri", "mongodb://localhost:27017/CovidDB") \
        .config("spark.mongodb.output.uri", "mongodb://localhost:27017/CovidDB") \
        .getOrCreate()
spark

### Coleta de Dados Brutos

In [3]:
#Rodar de segunda a sexta após as 16:00 devido disponibilização dos dados de vacinação.
#Não havendo dados de vacinação não haverá coleta de dados deste.

# url = "https://www.seade.gov.br/wp-content/uploads/2021/06/Dados-covid-19-estado.csv" #antigo
url = 'https://www.seade.gov.br/wp-content/uploads/coronavirus-files/Dados-covid-19-estado.csv' # novo

# print(url)
print('Realizando Extratacao Raw Data COVID')
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('Dados-covid-19-estado.csv'), inferSchema=True, header=True, sep=';')
print('FIM da Extratacao Raw Data COVID')

print('Realizando Gravacao Raw Data COVID')
#Inicio gravação dos dados da covid via mongodb
df.write.format("com.mongodb.spark.sql.DefaultSource")\
        .option("uri", "mongodb://ferhspz:27017/CovidDB.raw_data_covid_sp")\
        .mode("overwrite")\
        .save()
#Fim gravação dos dados da covid via mongodb
print('FIM da Gravacao Raw Data COVID')

#####Dados Vacianação######
#Rodar de segunda a sexta após as 16:00 devido disponibilização dos dados de vacinação.
#Não havendo dados de vacinação não haverá coleta de dados deste.

hora = str(datetime.now().hour) + str(datetime.now().minute)
dataAtual = date.today()

if(int(hora) < 1600):
    dataAtual = dataAtual - timedelta(1)
    print(f'Coleta do dia anterior - {dataAtual}')
    
mesAtual = str(dataAtual.month).rjust(2, '0')
anoAtual = str(dataAtual.year).rjust(4, '0')
diaAtual = str(dataAtual.day).rjust(2, '0')
dataAtualFmt = anoAtual+mesAtual+diaAtual

print('Realizando Extratacao Raw Data Vacinacao')
# Trata Url da evolução da vacinação
urlVac = f"https://www.saopaulo.sp.gov.br/wp-content/uploads/{anoAtual}/{mesAtual}/{dataAtualFmt}_evolucao_aplicacao_doses.csv"

arqVac = urlVac[58::1]

print(urlVac)
print(arqVac)

spark.sparkContext.addFile(urlVac)

# Create an expected schema
columns = StructType([StructField('Data', StringType(), True),
                        StructField('Dose', StringType(), True),
                        StructField('Qtde', IntegerType(), True)])

dfv = spark.read.csv(SparkFiles.get(arqVac), schema=columns, header=True, sep=';')    
print('FIM da Extratacao Raw Data Vacinacao')

print('Realizando Gravacao Raw Data COVID')
#Inicio gravação dos dados da covid via mongodb
dfv.write.format("com.mongodb.spark.sql.DefaultSource")\
        .option("uri", "mongodb://ferhspz:27017/CovidDB.raw_data_vaci_sp")\
        .mode("overwrite")\
        .save()
#Fim gravação dos dados da covid via mongodb
print('FIM da Gravacao Raw Data COVID')


print('Lendo Base Raw Data COVID')
df1 = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
                .option("uri", "mongodb://ferhspz:27017/CovidDB.raw_data_covid_sp")\
                .load()
df1.printSchema()
print('Leitura Concluida')


print('Lendo Base Raw Data COVID')
df2 = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
                .option("uri", "mongodb://ferhspz:27017/CovidDB.raw_data_vaci_sp")\
                .load()
df2.printSchema()
print('Leitura Concluida')

Realizando Extratacao Raw Data COVID
FIM da Extratacao Raw Data COVID
Realizando Gravacao Raw Data COVID
FIM da Gravacao Raw Data COVID
Coleta do dia anterior - 2022-09-30
Realizando Extratacao Raw Data Vacinacao
https://www.saopaulo.sp.gov.br/wp-content/uploads/2022/09/20220930_evolucao_aplicacao_doses.csv
20220930_evolucao_aplicacao_doses.csv
FIM da Extratacao Raw Data Vacinacao
Realizando Gravacao Raw Data COVID
FIM da Gravacao Raw Data COVID
Lendo Base Raw Data COVID
root
 |-- Casos: integer (nullable = true)
 |-- Casos por dia: integer (nullable = true)
 |-- Data: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- �bitos por dia: integer (nullable = true)

Leitura Concluida
Lendo Base Raw Data COVID
root
 |-- Data: string (nullable = true)
 |-- Dose: string (nullable = true)
 |-- Qtde: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)

Leitura Concluida
